### Importar los datos:

In [1]:
import pandas as pd

df_nombres = pd.read_csv('historico-nombres.csv', dtype= {'cantidad': 'int16', 'anio': 'int16'})

In [25]:
print('Valor máximo en cantidad:', max(df_nombres['cantidad'])) #Puedo usar int16
print('\nHead de los datos: \n', df_nombres.head())

Valor máximo en cantidad: 7939

Head de los datos: 
         nombre  cantidad  anio
0        Maria       314  1922
1         Rosa       203  1922
2         Jose       163  1922
3  Maria Luisa       127  1922
4       Carmen       117  1922


### Normalización de valores:

Busco ahora normalizar los nombres, es decir, que no haya inconsistencias de escritura (**tarda en ejecutarse**):

In [2]:
def normalizar_nombres(nombre):
    # 1) Formateo a String
    nombre = str(nombre)
    
    # 2) Limpiar caracteres especiales y/o números
    nombre = ''.join(filter(lambda c: c.isalpha() or c.isspace(), nombre))
    
    # 3) Limpiar espacios en el principio, final y/o de más
    nombre = ' '.join(nombre.split())
    
    # 4) Normalizar la capitalización
    nombre = nombre.title()
    
    # 5) Normalizar las tildes
    sin_tilde = {
    'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
    'à': 'a', 'è': 'e', 'ì': 'i', 'ò': 'o', 'ù': 'u',
    'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
    'À': 'A', 'È': 'E', 'Ì': 'I', 'Ò': 'O', 'Ù': 'U',
    }
    
    for tilde, sin_tilde in sin_tilde.items():
        nombre = nombre.replace(tilde, sin_tilde)
        
    return nombre

# Aplicar la función normalizar_nombres a la columna 'nombre'
df_nombres['nombre'] = df_nombres['nombre'].apply(normalizar_nombres)

### Transformación de datos:

Pivoteo la tabla de manera de obtener para cada nombre la cantidad de registros por año en cada fila.

In [4]:
nombres_pivot = df_nombres.pivot_table(index='nombre', columns='anio', values='cantidad', aggfunc='sum')

Algunos nombres para ojear un poco la nueva tabla:

In [21]:
nombres = ['Diego Armando', 'Diego', 'Lionel', 'Catalina', 'Santiago', 'Juan Carlos']
posiciones = []

for n in nombres:
    posiciones.append(nombres_pivot.index.get_loc(n))

print(nombres_pivot.iloc[posiciones, nombres_pivot.columns.get_loc(1984):nombres_pivot.columns.get_loc(1992)])

anio             1984    1985    1986    1987    1988    1989    1990    1991
nombre                                                                       
Diego Armando   537.0   536.0  1501.0   667.0   560.0   478.0   622.0   417.0
Diego           331.0   376.0   425.0   388.0   380.0   358.0   361.0   429.0
Lionel           21.0    28.0    34.0    21.0    30.0    19.0    16.0    22.0
Catalina        207.0   215.0   245.0   238.0   287.0   267.0   370.0   377.0
Santiago        757.0   946.0  1055.0  1124.0  1134.0  1222.0  1254.0  1416.0
Juan Carlos    3171.0  2988.0  3210.0  3169.0  2940.0  2594.0  2438.0  2497.0


Cabe aclarar que en la normalización, los nombres compuestos los mantuve como nombres compuestos. Se ve la diferencia en los registros entre Diego y Diego Armando.

Lo siguiente es armar una nueva tabla sólo con los nombres que en algún año hayan rankeado en una posición relevante de popularidad, es decir, aquellos que quiero que aparezcan en la visualización final.

**1. Necesito una nueva tabla de porcentajes para cada nombre en cada año.**

In [26]:
totales_años = nombres_pivot.sum()
print(totales_años)

anio
1922     23667.0
1923     30289.0
1924     39075.0
1925     48328.0
1926     58350.0
          ...   
2011    450370.0
2012    950740.0
2013    900028.0
2014    881267.0
2015    793166.0
Length: 94, dtype: float64


Calculo los porcentajes y lo guardo en un nuevo DataFrame, esto tarda **muchísimo** en ejecutarse, así que luego de ejecutarlo una vez guarde una versión reducida.

```Python
porcentajes = pd.DataFrame()
for columna in nombres_pivot.columns:
    porcentajes[columna] = nombres_pivot[columna].apply(lambda x: (x / totales_años[columna]) * 100)
   
umbral = 0.005 #Arbitrario
# Filtrar las filas que tengan al menos una columna que supere el umbral
porcentajes_filtrados = porcentajes[porcentajes.gt(umbral).any(axis=1)]
# Guardo este esta tabla de porcentajes reducida
porcentajes_filtrados.to_csv('porcentajes_reducida.csv')
```

**2. Reduzco la tabla a los nombres más populares**

In [70]:
porcentajes_filtrados = pd.read_csv('porcentajes_reducida.csv', index_col='nombre')

En la animación que quiero crear, habrá ocho nombres a la vista en todo momento.

Busco para que umbrales obtengo al menos ocho nombres en cada año. Esto también sirve para reducir la cantidad de nombres a los que luego voy a tener que asignarles género para la visualización.

In [71]:
import numpy as np

umbrales = np.arange(0.015, 3, 0.015)
cant_supera_umbral = pd.DataFrame(index=umbrales, columns=nombres_pivot.columns)

for año in nombres_pivot.columns:
    for umbral in umbrales:
        cant_supera_umbral.at[umbral, año] = (porcentajes_filtrados[str(año)] >= umbral).sum()

In [72]:
cant_supera_umbral.index.name = 'umbral'
cant_supera_umbral.head()

anio,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
umbral,,,,,,,,,,,,,,,,,,,,,
0.015,876,784,787,684,735,678,710,669,655,650,...,677,652,623,731,613,585,578,564,527,522
0.030,351,328,334,330,322,319,307,322,316,297,...,253,250,248,302,249,242,236,232,226,225
0.045,246,233,229,220,204,207,205,202,191,190,...,146,142,140,170,147,145,136,141,138,139
0.060,178,167,171,166,155,163,147,143,150,140,...,97,102,99,115,107,111,103,102,99,101
0.075,150,136,132,134,131,133,126,123,125,122,...,70,71,75,92,80,87,81,80,77,77


In [50]:
umbrales_posibles = cant_supera_umbral[(cant_supera_umbral >= 8).all(axis=1)]
print(umbrales_posibles)

anio   1922 1923 1924 1925 1926 1927 1928 1929 1930 1931  ... 2006 2007 2008  \
umbral                                                    ...                  
0.015   199  197  206  196  207  207  209  205  209  209  ...  171  169  171   
0.030   174  173  179  179  181  179  180  180  183  178  ...  133  132  130   
0.045   158  163  161  165  157  161  161  158  155  157  ...  106  110  109   
0.060   146  144  146  147  141  149  138  139  142  135  ...   86   92   91   
0.075   138  126  129  129  127  130  124  123  124  121  ...   65   67   73   
0.090   119  113  118  115  109  107  103  102   99  101  ...   56   57   57   
0.105    98  103   90   94   93   88   89   85   83   76  ...   49   46   50   
0.120    77   83   81   82   83   73   75   71   70   69  ...   40   42   41   
0.135    71   72   72   69   70   65   63   63   59   61  ...   34   33   36   
0.150    63   64   65   59   58   57   52   53   51   47  ...   32   31   32   
0.165    54   58   56   51   49   47   4

Me voy a quedar con el umbral máximo para reducir la cantidad de nombres

In [73]:
umbral_max = max(umbrales_posibles.index)
print(umbral_max)

0.18


In [74]:
porcentajes_filtrados = porcentajes_filtrados[porcentajes_filtrados.gt(umbral_max).any(axis=1)]

**3. Asignar género a los nombres**

Investigando por internet, pareciera ser que los datos en algún momento tuvieron el género asignado para cada nombre, pero fueron actualizados sin esa columna. Para poder asignarlos, voy a usar dos reglas muy generales que sé que van a generar casos incorrectos, después lo corrijo manualmente.

In [75]:
porcentajes_filtrados['genero'] = None

In [80]:
porcentajes_filtrados

,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,genero
nombre,,,,,,,,,,,,,,,,,,,,,
Abril,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.128788,0.095811,0.096196,0.076917,0.070387,0.056167,0.045443,0.045616,0.043497,None
Agustin,0.046478,0.059428,0.074216,0.053799,0.053128,0.067433,0.050199,0.059721,0.062950,0.060103,...,0.253618,0.246208,0.266984,0.245586,0.274663,0.197215,0.165550,0.170323,0.170204,None
Agustina,0.080281,0.075935,0.053743,0.053799,0.053128,0.060409,0.038525,0.059721,0.043979,0.041123,...,0.179540,0.163755,0.168558,0.145959,0.147878,0.103393,0.095108,0.093842,0.092162,F
Alberto,0.185913,0.277328,0.217530,0.219335,0.243359,0.266921,0.220642,0.202446,0.244901,0.222222,...,0.003110,0.002298,0.003601,0.001831,0.002664,0.002104,0.002444,0.001248,0.001261,M
Alma,0.008451,0.009905,0.005118,0.002069,NaN,0.005619,0.001167,0.005061,0.001725,0.002372,...,0.062910,0.072541,0.146952,0.142663,0.197171,0.178598,0.171106,0.153415,0.151923,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Valentino,NaN,NaN,NaN,0.002069,NaN,0.004215,NaN,0.001012,0.001725,0.003163,...,0.200604,0.201965,0.266469,0.304922,0.306859,0.264426,0.255548,0.306717,0.295146,M
Vicente,0.190138,0.151870,0.176583,0.163466,0.169666,0.148914,0.180950,0.136651,0.143147,0.136813,...,0.025305,0.037491,0.048184,0.054941,0.065946,0.065633,0.063554,0.074552,0.096323,None
Victor Hugo,0.004225,0.009905,0.025592,0.014484,0.017138,0.026692,0.005837,0.014171,0.031906,0.024516,...,0.008765,0.008331,0.006173,0.007692,0.004441,0.003576,0.003555,0.002837,0.002269,M


In [79]:
for nombre in porcentajes_filtrados.index:
    splitted = nombre.split()
    for s in splitted:
        if s[-1] == 'a':
            porcentajes_filtrados.at[nombre, 'genero'] = 'F'
        elif s[-1] == 'o':
            porcentajes_filtrados.at[nombre, 'genero'] = 'M'

In [ ]:
porcentajes_filtrados.to_csv('nombres_mas_populares.csv')

Exportando estos datos, y visualizandolos en Flourish voy a analizar los casos especiales como Bautista, en caso de que aparezcan en la visualización otros nombres mal clasificados o sin clasificar, lo corrijo manualmente.

Se puede ver en pruebas antes de la animación definitiva que el nombre Leonel Hernan tiene un pico de registros muy poco fiable, pasando de 9 registros en 1974 a  7939 al año siguiente y bajando a 10 en 1976. Elimino la fila.

In [ ]:
porcenajes_filtrados_final = pd.read_csv('nombres_mas_populares_final.csv')

### Resultado final

In [14]:
from IPython.display import HTML
html_code = """
<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/17097755">
    <script src="https://public.flourish.studio/resources/embed.js"></script>
</div>
"""
display(HTML(html_code))